In [106]:

# importacion general de librerias y de visualizacion (matplotlib y seaborn)# import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

# Data Tp1

In [107]:
postulaciones = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_4_postulaciones.csv', low_memory=False)
educacion = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv', low_memory=False)
genero_y_edad = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
vistas = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_3_vistas.csv', low_memory=False)
avisos_online = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_5_avisos_online.csv', low_memory=False)
detale = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/datos_navent_fiuba/fiuba_6_avisos_detalle.csv', low_memory=False)

# Data hasta 15 de abril

In [108]:
postulaciones_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_4_postulaciones.csv', low_memory=False)
educacion_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_1_postulantes_educacion.csv', low_memory=False)
genero_y_edad_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
vistas_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_3_vistas.csv', low_memory=False)
avisos_online_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_5_avisos_online.csv', low_memory=False)
detale_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/hasta_15_abril/fiuba_6_avisos_detalle.csv', low_memory=False)

### Hasta aca seria el entrenamiento

# Data post 15 de abril

In [109]:
educacion_post_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/desde_15_abril/fiuba_1_postulantes_educacion.csv', low_memory=False)
genero_y_edad_post_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)
vistas_post_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/desde_15_abril/fiuba_3_vistas.csv', low_memory=False)
detale_post_abril = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/desde_15_abril/fiuba_6_avisos_detalle.csv', low_memory=False)

# Test final

In [110]:
test_final = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/test_final_100k.csv', low_memory=False)

## Tratando de unificar

Trato de juntar la info de toda la parte que seria de entrenamiento, es decir, de lo que ya sabemos como se dieron las postulaciones

In [111]:
lista_educacion = [educacion, educacion_abril]
educacion_total = pd.concat(lista_educacion)

lista_genero = [genero_y_edad, genero_y_edad_abril]
genero_total = pd.concat(lista_genero)

lista_vistas = [vistas, vistas_abril]
vistas_total = pd.concat(lista_vistas)

detalle_faltante = pd.read_csv('/home/rodrigo/ENV/Data/Tp2/fiuba_6_avisos_detalle_missing_nivel_laboral.csv', low_memory=False)
lista_detalle = [detale, detale_abril]
detalle_total = pd.concat(lista_detalle)

lista_postulaciones = [postulaciones, postulaciones_abril]
postulaciones_total = pd.concat(lista_postulaciones)

## Saco duplicados

In [112]:
educacion_total = educacion_total.drop_duplicates(subset=['idpostulante','nombre', 'estado'])

#Pongo la condicion de que se repita en los 3 campos porque podemos tener postulantes con varias instancias de educacion

In [113]:
genero_total = genero_total.drop_duplicates(subset='idpostulante')

In [114]:
vistas_total = vistas_total.drop_duplicates(subset=['idAviso', 'timestamp', 'idpostulante'])

In [115]:
detalle_total = detalle_total.drop_duplicates(subset='idaviso')

In [116]:
postulaciones_total = postulaciones_total.drop_duplicates(subset=['idaviso', 'idpostulante'])

# Analisis de los datos

## Genero y edad

In [117]:
genero_total.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [118]:
genero_total.isnull().sum()

idpostulante           0
fechanacimiento    19644
sexo                   0
dtype: int64

In [119]:
genero_total['fechanacimiento'] = pd.to_datetime(genero_total['fechanacimiento'], errors = 'coerce')

El coerce es para que las fechas que no pueda transformar las pase a null

In [120]:
genero_total.isnull().sum()

idpostulante           0
fechanacimiento    19671
sexo                   0
dtype: int64

In [121]:
genero_total = genero_total.dropna()

In [122]:
genero_total.isnull().sum()

idpostulante       0
fechanacimiento    0
sexo               0
dtype: int64

In [123]:
genero_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388475 entries, 0 to 297745
Data columns (total 3 columns):
idpostulante       388475 non-null object
fechanacimiento    388475 non-null datetime64[ns]
sexo               388475 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 11.9+ MB


Efectivamente tenemos las fechas pasadas a date time

In [124]:
now = pd.Timestamp(datetime.datetime.now())
genero_total['age'] = (now - genero_total['fechanacimiento']).astype('timedelta64[Y]').astype(int)

In [125]:
genero_total.head()

,idpostulante,fechanacimiento,sexo,age
0,NM5M,1970-12-03,FEM,47
1,5awk,1962-12-04,FEM,55
2,ZaO5,1978-08-10,FEM,39
3,NdJl,1969-05-09,MASC,49
4,eo2p,1981-02-16,MASC,37


In [126]:
genero_total['age'].max()

242

In [127]:
genero_total['age'].min()

9

Voy a filtrar las edades entre 18 y 125 anios

In [128]:
genero_total = genero_total[genero_total['age'] <= 125]
genero_total = genero_total[genero_total['age'] >= 18]

print (genero_total['age'].max())
print (genero_total['age'].min())

116
18


In [129]:
genero_total['sexo'].value_counts()

FEM           203723
MASC          183604
NO_DECLARA       943
0.0                1
Name: sexo, dtype: int64

Tenemos un valor invalido, 0.0

In [130]:
genero_total = genero_total[genero_total['sexo'] != '0.0']

In [131]:
genero_total['sexo'].value_counts()

FEM           203723
MASC          183604
NO_DECLARA       943
Name: sexo, dtype: int64

### Hay edades atipicas con solo 1 o 2 personas. Podrian sacarse, pero por ahora lo dejo

## Educacion

In [132]:
educacion_total.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [133]:
educacion_total.isnull().sum()

idpostulante    0
nombre          0
estado          0
dtype: int64

In [134]:
educacion_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568612 entries, 0 to 407452
Data columns (total 3 columns):
idpostulante    568612 non-null object
nombre          568612 non-null object
estado          568612 non-null object
dtypes: object(3)
memory usage: 17.4+ MB


In [135]:
educacion_total['nombre'].value_counts()

Universitario        207752
Secundario           199375
Terciario/Técnico     92632
Otro                  43225
Posgrado              16822
Master                 8299
Doctorado               507
Name: nombre, dtype: int64

In [136]:
educacion_total['estado'].value_counts()

Graduado      370213
En Curso      149388
Abandonado     49011
Name: estado, dtype: int64

## Vistas

In [137]:
vistas_total.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [138]:
vistas_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6682624 entries, 0 to 5758685
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 203.9+ MB


In [139]:
vistas_total.isnull().sum()

idAviso         0
timestamp       0
idpostulante    0
dtype: int64

In [140]:
vistas_total['timestamp'] = pd.to_datetime(vistas_total['timestamp'])

In [141]:
vistas_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6682624 entries, 0 to 5758685
Data columns (total 3 columns):
idAviso         int64
timestamp       datetime64[ns]
idpostulante    object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 203.9+ MB


Tenemos los timestamp en formato date time

## Detalle

In [142]:
detalle_total.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [143]:
detalle_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19358 entries, 0 to 18291
Data columns (total 11 columns):
idaviso                 19358 non-null int64
idpais                  19358 non-null int64
titulo                  19358 non-null object
descripcion             19358 non-null object
nombre_zona             19358 non-null object
ciudad                  70 non-null object
mapacalle               1373 non-null object
tipo_de_trabajo         19358 non-null object
nivel_laboral           19358 non-null object
nombre_area             19358 non-null object
denominacion_empresa    19352 non-null object
dtypes: int64(2), object(9)
memory usage: 1.8+ MB


In [144]:
detalle_total.isnull().sum()

idaviso                     0
idpais                      0
titulo                      0
descripcion                 0
nombre_zona                 0
ciudad                  19288
mapacalle               17985
tipo_de_trabajo             0
nivel_laboral               0
nombre_area                 0
denominacion_empresa        6
dtype: int64

In [145]:
detalle_total['idpais'].value_counts()

1    19358
Name: idpais, dtype: int64

#### Como todos son del mismo pais saco ese feature

In [146]:
del detalle_total['idpais']
del detalle_total['descripcion']
del detalle_total['mapacalle']

In [147]:
detalle_total.head()

,idaviso,titulo,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,Chofer de taxi,Capital Federal,NaN,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


## Postulaciones

In [148]:
postulaciones_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 201.5+ MB


In [149]:
postulaciones_total.isnull().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [150]:
postulaciones_total['fechapostulacion'] = pd.to_datetime(postulaciones_total['fechapostulacion'])

In [151]:
postulaciones_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 201.5+ MB


### Hay que generar algun df del estilo idpostulatante, idaviso, sepostulo junto con toda la info de los avisos y los postulantes

Para eso tengo que agarrar ids de postulante e ids de aviso random para luego ver si esa postulacion se dio o no

Pero antes para eso unifico info de los postulantes. La de los avisos ya esta toda en detalle


In [152]:
genero_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388270 entries, 0 to 297745
Data columns (total 4 columns):
idpostulante       388270 non-null object
fechanacimiento    388270 non-null datetime64[ns]
sexo               388270 non-null object
age                388270 non-null int64
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 14.8+ MB


In [153]:
educacion_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568612 entries, 0 to 407452
Data columns (total 3 columns):
idpostulante    568612 non-null object
nombre          568612 non-null object
estado          568612 non-null object
dtypes: object(3)
memory usage: 17.4+ MB


In [154]:
postulantes = genero_total.merge(educacion_total, on='idpostulante', how='inner')

In [155]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558955 entries, 0 to 558954
Data columns (total 6 columns):
idpostulante       558955 non-null object
fechanacimiento    558955 non-null datetime64[ns]
sexo               558955 non-null object
age                558955 non-null int64
nombre             558955 non-null object
estado             558955 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 29.9+ MB


In [156]:
detalle_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19358 entries, 0 to 18291
Data columns (total 8 columns):
idaviso                 19358 non-null int64
titulo                  19358 non-null object
nombre_zona             19358 non-null object
ciudad                  70 non-null object
tipo_de_trabajo         19358 non-null object
nivel_laboral           19358 non-null object
nombre_area             19358 non-null object
denominacion_empresa    19352 non-null object
dtypes: int64(1), object(7)
memory usage: 1.3+ MB


In [157]:
postulantes.head()

,idpostulante,fechanacimiento,sexo,age,nombre,estado
0,NM5M,1970-12-03,FEM,47,Secundario,Graduado
1,5awk,1962-12-04,FEM,55,Universitario,Graduado
2,5awk,1962-12-04,FEM,55,Terciario/Técnico,Graduado
3,ZaO5,1978-08-10,FEM,39,Terciario/Técnico,Graduado
4,ZaO5,1978-08-10,FEM,39,Universitario,Abandonado


In [158]:
detalle_total.head()

,idaviso,titulo,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,Chofer de taxi,Capital Federal,NaN,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [159]:
postulantes.head()

,idpostulante,fechanacimiento,sexo,age,nombre,estado
0,NM5M,1970-12-03,FEM,47,Secundario,Graduado
1,5awk,1962-12-04,FEM,55,Universitario,Graduado
2,5awk,1962-12-04,FEM,55,Terciario/Técnico,Graduado
3,ZaO5,1978-08-10,FEM,39,Terciario/Técnico,Graduado
4,ZaO5,1978-08-10,FEM,39,Universitario,Abandonado


In [160]:
detalle_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19358 entries, 0 to 18291
Data columns (total 8 columns):
idaviso                 19358 non-null int64
titulo                  19358 non-null object
nombre_zona             19358 non-null object
ciudad                  70 non-null object
tipo_de_trabajo         19358 non-null object
nivel_laboral           19358 non-null object
nombre_area             19358 non-null object
denominacion_empresa    19352 non-null object
dtypes: int64(1), object(7)
memory usage: 1.3+ MB


In [199]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558955 entries, 0 to 558954
Data columns (total 6 columns):
idpostulante       558955 non-null object
fechanacimiento    558955 non-null datetime64[ns]
sexo               558955 non-null object
age                558955 non-null int64
nombre             558955 non-null object
estado             558955 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 29.9+ MB


In [208]:
    sample_avisos = detalle_total.sample(n=9000)
    sample_postulantes = postulantes.sample(n=9000)
    idspostulantes = sample_postulantes['idpostulante'].reset_index(drop=True)
    idsavisos = sample_avisos['idaviso'].reset_index(drop=True)
    d = {'idpostulante':idspostulantes,'idaviso':idsavisos}
    aux = pd.DataFrame(d)

In [209]:
prueba.head()

,idaviso,idpostulante,sepostulo
0,1112463470,mzbpMp9,0
1,1112280978,JBdddE5,0
2,1112303957,A3XPGr5,0
3,1112392676,EzpWLk4,0
4,1112369740,jkj9jWr,0


In [168]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 2 columns):
idaviso         9000 non-null int64
idpostulante    9000 non-null object
dtypes: int64(1), object(1)
memory usage: 140.7+ KB


In [169]:
prueba['sepostulo'] = 0

In [170]:
prueba.head()

,idaviso,idpostulante,sepostulo
0,1112463470,mzbpMp9,0
1,1112280978,JBdddE5,0
2,1112303957,A3XPGr5,0
3,1112392676,EzpWLk4,0
4,1112369740,jkj9jWr,0


In [171]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 3 columns):
idaviso         9000 non-null int64
idpostulante    9000 non-null object
sepostulo       9000 non-null int64
dtypes: int64(2), object(1)
memory usage: 211.0+ KB


In [172]:
postulaciones_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 201.5+ MB


In [194]:
postulaciones_total.drop_duplicates(subset='idaviso').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20231 entries, 0 to 4808006
Data columns (total 3 columns):
idaviso             20231 non-null int64
idpostulante        20231 non-null object
fechapostulacion    20231 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 632.2+ KB


In [175]:
prueba_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9000 entries, 1630362 to 2584848
Data columns (total 3 columns):
idaviso             9000 non-null int64
idpostulante        9000 non-null object
fechapostulacion    9000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 281.2+ KB


In [177]:

del prueba_postulaciones['fechapostulacion']
prueba_postulaciones.head()

,idaviso,idpostulante
1630362,1112316288,rm2wKz6
4149413,1112468058,xkaO1N6
2426716,1112281048,EzOZZwo
518708,1112337868,bO4r2DY
182224,1112145679,4rm90qR


In [181]:
prueba_postulaciones['sepostulo'] = 1

In [184]:
prueba_postulaciones = prueba_postulaciones.reset_index(drop=True)
prueba_postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112316288,rm2wKz6,1
1,1112468058,xkaO1N6,1
2,1112281048,EzOZZwo,1
3,1112337868,bO4r2DY,1
4,1112145679,4rm90qR,1


In [185]:
prueba.head()

,idaviso,idpostulante,sepostulo
0,1112463470,mzbpMp9,0
1,1112280978,JBdddE5,0
2,1112303957,A3XPGr5,0
3,1112392676,EzpWLk4,0
4,1112369740,jkj9jWr,0


In [227]:
postulaciones_prueba = postulaciones_total.sample(n=9000)
postulaciones_prueba['sepostulo'] = 1
del postulaciones_prueba['fechapostulacion']

In [228]:
lista = [prueba, postulaciones_prueba]
entrenamiento = pd.concat(lista)

In [229]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 3063078
Data columns (total 3 columns):
idaviso         18000 non-null int64
idpostulante    18000 non-null object
sepostulo       18000 non-null int64
dtypes: int64(2), object(1)
memory usage: 562.5+ KB


In [230]:
entrenamiento = entrenamiento.drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')

In [231]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18000 entries, 0 to 3063078
Data columns (total 3 columns):
idaviso         18000 non-null int64
idpostulante    18000 non-null object
sepostulo       18000 non-null int64
dtypes: int64(2), object(1)
memory usage: 562.5+ KB


In [233]:
entrenamiento = entrenamiento.merge(postulantes, on='idpostulante', how='inner')

In [235]:
entrenamiento.drop_duplicates(subset='idpostulante').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16762 entries, 0 to 32177
Data columns (total 8 columns):
idaviso            16762 non-null int64
idpostulante       16762 non-null object
sepostulo          16762 non-null int64
fechanacimiento    16762 non-null datetime64[ns]
sexo               16762 non-null object
age                16762 non-null int64
nombre             16762 non-null object
estado             16762 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 1.2+ MB


In [236]:
entrenamiento = entrenamiento.merge(detalle_total, on='idaviso', how='inner')

In [241]:
entrenamiento = entrenamiento.drop_duplicates(subset='idpostulante')

In [242]:
entrenamiento.head()

,idaviso,idpostulante,sepostulo,fechanacimiento,sexo,age,nombre,estado,titulo,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112463470,mzbpMp9,0,1994-07-27,FEM,23,Secundario,En Curso,Desarrollador web .Net,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Programación,HV Talent
3,1112280978,JBdddE5,0,1997-02-21,FEM,21,Secundario,Graduado,Analista de Presupuestos,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Finanzas,Claro Argentina
4,1112303957,A3XPGr5,0,1985-03-16,MASC,33,Universitario,En Curso,CALDERERO / ARMADOR,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Producción,Adecco -Región GBA SUR
8,1112392676,EzpWLk4,0,1989-03-22,MASC,29,Secundario,Graduado,VENDEDOR ON TRADE - SECTOR GASTRONÓMICO,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Ventas,MUNTANER RRHH
10,1112369740,jkj9jWr,0,1992-06-02,MASC,26,Terciario/Técnico,En Curso,A- Promovendedora zona Sur,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Ventas,Grupo Gestión


#### Yo no creo que esto sea representativo pero lo exporto para empezar a probar algo.

In [243]:
entrenamiento.to_csv('pruebaMala.csv', index=False)